# Importing Libs

In [1]:
import pandas as pd
import numpy as np
import glob
import os
pd.set_option('display.max_rows', None)

# Importing Dataset

In [2]:
df = pd.read_csv("User-DealWOutActivity.csv")
df['BizDomainId'] = df['BizDomainId'].str.lower()
df_final = pd.read_csv("df_train.csv" , index_col=0)
BizIdAll = set(df['BizDomainId'])
BizIdNow = set(df_final['BizDomainId'])
churnedBizId= list(BizIdAll.difference(BizIdNow))
print('churned BizIds:',churnedBizId[:5] , 'number of churned:' , len(churnedBizId))
df.head()

churned BizIds: ['ab0ae364-0fb5-4266-99a0-5748163b2808', '07c40027-f9a1-4fce-b74d-006580bb0c9f', '114b639d-c497-49f9-bdcc-6bb2d8bd9485', 'b1d6f37e-51b7-4c95-86f3-1df4bb73baea', '2a96e22a-5a4f-4605-8273-8bf9915a928d'] number of churned: 2384


,BizDomainId,Date,BoughtUser,AddedUsers,ActiveUsers,NumberOfDealsOnTheDate,TotalDealsWithoutActivity,NumberOfActivitiesOnTheDate
0,16c6651d-b6ba-47b2-b231-5fadd9b3d6c2,4/10/2022,3,3,1,0,9,0
1,d4e3162a-7fd2-490a-bd68-fc5c3d6dd464,4/10/2022,7,7,6,1,3,198
2,d7bf85aa-ee4c-4be3-916a-89ad83638c21,4/10/2022,3,2,1,0,1,0
3,579aa04a-7ff3-4a1a-9d56-b10c105ca98c,4/10/2022,1,1,1,7,95,0
4,b440ee60-ea75-4bdf-9542-e53501f616f2,4/10/2022,2,2,0,0,0,0


In [3]:
print("number of bizdomains:",len(df['BizDomainId'].unique()))

number of bizdomains: 4150


# Feature 2 & 3: AddedUsers/BoughtUsers & ActiveUsers/AddedUsers

In [4]:
df['AddedUsers/BoughtUser'] = df['AddedUsers']/df['BoughtUser']
df['ActiveUsers/AddedUsers'] = df['ActiveUsers']/df['AddedUsers']
df['ActiveUsers/AddedUsers'] = df['ActiveUsers/AddedUsers'].fillna(0)

In [5]:
df_final.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?
0,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1,0.0
29,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0,1.0
161,00428596-e168-49ce-8428-7949cd2215ea,0.822599,0,0.0
175,004c3ec1-da71-494d-9adb-8a4e660f9257,0.082940,0,1.0
1702,00d34eb3-bee5-4846-af99-46abcb9feced,0.407162,0,0.0


In [6]:
df_train = df.groupby('BizDomainId')[['AddedUsers/BoughtUser' , 'ActiveUsers/AddedUsers']].agg('mean')
df_train = df_train.reset_index()
df_final = pd.merge(df_final, df_train, how='right' , left_on = 'BizDomainId' , right_on = 'BizDomainId')
df_final['BizIsExpired?'] = df_final['BizIsExpired?'].fillna('others')
df_final['BizActive?'] = df_final['BizActive?'].fillna('others')
df_final.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000


# Importing file 1,2,3 for checking stuff as df_file_checking
For checking number of deals and activities, this dataframe (the version which is not cleaned) is needed.

In [7]:
path = r"files" # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in all_files:
    temp = pd.read_csv(filename, index_col=None, header=0)
    li.append(temp)

frame = pd.concat(li, axis=0, ignore_index=True)

df_file_checking = frame.rename(columns={'IsDeleted':'IsDeletedDeal','Id':'DealId' , 'Id.1':'ActivityId','IsDeleted.1':'IsDeletedActivity','RegisterTime':'RegisterTimeDeal',
                                                                                    'Status':'StatusDeal','RegisterDate':'RegisterDateActivity','IsDone':'IsDoneActivity',
                                                                                    'DoneDate':'DoneDateActivity'})
df_file_checking['DealId']=df_file_checking['DealId'].str.lower()
df_file_checking['DoneDateActivity'] = np.where(df_file_checking['IsDoneActivity']==False , np.nan ,df_file_checking['DoneDateActivity'])
df_file_checking['DealFinishTime']=np.where(df_file_checking['StatusDeal']==1 , df_file_checking['ChangeToWonTime'] , 
                                np.where(df_file_checking['StatusDeal']==2 , df_file_checking['ChangeToLossTime'] , None))

biz1=pd.read_csv(r"Bizdomain1.csv")
biz2=pd.read_csv(r"Bizdomain2.csv")

# complete bizdomains (appending 2 sheets of Bizdomain excel)
biz=pd.concat([biz1,biz2])
biz['DealId']=biz['DealId'].str.lower()
biz['BizDomainId']=biz['BizDomainId'].str.lower()

#test for merging of complete biz and 100 rows of file01 which was calcultaed for interval calculs
df_file_checking = pd.merge(df_file_checking, biz, how='left')
# df_file_checking.head()

ExpireDate=pd.read_csv(r"ExpireDate.csv").rename(columns={'تاریخ ثبت نام':'BizRegisterDate','تاریخ انقضا':'BizExpireDate' , 'اخرین تاریخ دسترسی':'BizAccessDate',
                                                                                    'یوزر خریداری شده':'BizBoughtUser','آیا فعال است':'BizActive?'})
ExpireDate['BizDomainId']=ExpireDate['BizDomainId'].str.lower()


df_file_checking = pd.merge(df_file_checking, ExpireDate, how='left')
df_file_checking['BizRegisterDate'] = pd.to_datetime(df_file_checking['BizRegisterDate'])
df_file_checking['BizExpireDate'] = pd.to_datetime(df_file_checking['BizExpireDate'])
df_file_checking['BizAccessDate'] = pd.to_datetime(df_file_checking['BizAccessDate'])

#Pay attention to export date
exportdate=pd.Timestamp('2022-7-31T12')
df_file_checking['BizIsExpired?']=np.where(df_file_checking['BizExpireDate']>=exportdate,0,1)
df_file_checking['IsDeletedDeal']=df_file_checking['IsDeletedDeal'].map({True: 1, False: 0})
df_file_checking['IsDeletedActivity']=df_file_checking['IsDeletedActivity'].map({True: 1, False: 0})
df_file_checking['IsDoneActivity']=df_file_checking['IsDoneActivity'].map({True: 1, False: 0})
df_file_checking.head()

,DealId,IsDeletedDeal,RegisterTimeDeal,StatusDeal,ChangeToWonTime,ChangeToLossTime,ActivityId,IsDeletedActivity,RegisterDateActivity,IsDoneActivity,DoneDateActivity,DealFinishTime,BizDomainId,BizRegisterDate,BizExpireDate,BizAccessDate,BizBoughtUser,BizActive?,BizIsExpired?
0,cef52513-8f12-4601-91f2-00002ce80633,0,2022-04-14 21:19:41.000,1,2022-04-14 21:20:16.480,2022-04-14 21:19:41.000,NaN,NaN,NaN,NaN,NaN,2022-04-14 21:20:16.480,314a88a8-6d18-4972-abf5-d701f54dee15,2022-01-12,2023-01-27,2022-07-30,3.0,1.0,0
1,5977d067-a390-4b9c-9d58-00003b1f11cf,0,2022-06-08 12:37:08.210,0,2022-06-08 12:37:08.210,2022-06-08 12:37:08.210,NaN,NaN,NaN,NaN,NaN,None,ca7dd0be-49a6-4b2e-ab21-3616d57e9571,2019-07-18,2022-09-26,2022-07-31,3.0,1.0,0
2,80f17f47-87a7-4259-b234-00004c7c6056,0,2022-06-24 19:03:12.280,1,2022-06-24 19:03:12.280,2022-06-24 19:03:12.280,NaN,NaN,NaN,NaN,NaN,2022-06-24 19:03:12.280,729423ab-c61c-4b98-8b49-56ac0349e7c9,2020-08-10,2022-11-15,2022-07-31,5.0,1.0,0
3,c6ad7211-1856-4129-b3b0-000053ded53b,0,2022-04-10 08:37:00.803,1,2022-04-10 08:37:00.803,2022-04-10 08:37:00.803,NaN,NaN,NaN,NaN,NaN,2022-04-10 08:37:00.803,9963048e-7339-4014-8e82-6115c8e337f4,2021-09-24,2022-11-02,2022-07-31,4.0,1.0,0
4,30322eb0-a08d-4b98-80e2-000067a52945,0,2022-06-13 08:11:20.647,0,2022-06-13 08:11:20.647,2022-06-13 08:11:20.647,NaN,NaN,NaN,NaN,NaN,None,986cdeeb-a977-4ffa-ade8-05fc2f566d19,2021-10-16,2022-10-31,2022-07-31,1.0,1.0,0


# Feature 4: Activities without Deal

##### Calculating total number of activities

In [8]:
path = r"files_cleaned" # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df_file = pd.concat(li, axis=0, ignore_index=True)

df_file['RegisterDateActivity'] = pd.to_datetime(df_file['RegisterDateActivity'])
starttime = pd.Timestamp('2022-3-21')
mask = df_file['RegisterDateActivity']>=starttime
df_file = df_file.loc[mask,:]
df_final = pd.merge(df_final, df_file.groupby('BizDomainId')['IsDeletedActivity'].count().to_frame().reset_index(),
         how='left').rename({'IsDeletedActivity':'NumberOfActivities'})

df_final = df_final.rename(columns={'BizdomainId':'BizDomainId' , 'IsDeletedActivity':'#Activities'})
df_final.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers,#Activities
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000,NaN
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397,4.0
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806,NaN
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307,102.0
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000,NaN


##### Calculating total number of activities without deal

In [9]:
df_woutdeal = pd.read_csv(r"ActivitiesWOutDeal.csv")
df_woutdeal['BizdomainId'] = df_woutdeal['BizdomainId'].str.lower()

df_final = pd.merge(df_final, df_woutdeal.groupby('BizdomainId')['ActivitiesWoutDeal'].sum().to_frame().reset_index(),
         how='left',left_on = 'BizDomainId' , right_on = 'BizdomainId').drop('BizdomainId',axis=1)
df_final.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers,#Activities,ActivitiesWoutDeal
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000,NaN,NaN
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397,4.0,26.0
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806,NaN,NaN
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307,102.0,200.0
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000,NaN,NaN


##### Filling th NaN values of ActivitiesWoutDeal when #Activities is not NaN with 0

In [10]:
mask = (~pd.isna(df_final['#Activities'])) & (pd.isna(df_final['ActivitiesWoutDeal']))
df_final.loc[mask , 'ActivitiesWoutDeal'] = 0
df_final['#Activities'] = df_final['#Activities'].fillna(0)+df_final['ActivitiesWoutDeal']
df_final['ActivitiesWoutDeal/#Activities'] = df_final['ActivitiesWoutDeal'] / df_final['#Activities']
del df_final['ActivitiesWoutDeal']
df_final.head(10)

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers,#Activities,ActivitiesWoutDeal/#Activities
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000,NaN,NaN
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397,30.0,0.866667
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806,NaN,NaN
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307,302.0,0.662252
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000,NaN,NaN
5,003a5023-c29c-47d7-b484-978733878bff,NaN,others,others,1.000000,0.000000,NaN,NaN
6,003d7420-f7c2-4780-9bff-b893620abf5a,NaN,others,others,0.500000,0.000000,NaN,NaN
7,00428596-e168-49ce-8428-7949cd2215ea,0.822599,0.0,0.0,1.000000,0.949580,13.0,0.000000
8,004506ef-869d-42d6-90fb-9290ec5e0c0b,NaN,others,others,0.200000,0.000000,NaN,NaN
9,0049986f-b17d-44ef-992f-88d478dd21b8,NaN,others,others,1.000000,0.211382,1.0,1.000000


# Feature 5: ActivitiesDelaysAvg

In [11]:
df_delay = pd.read_csv(r"ActivitiesDelays.csv")
df_delay['multiply'] = df_delay['AvgActivitiyDeal']*df_delay['#DelaidActivities']
df_delay['BizDomainId'] = df_delay['BizDomainId'].str.lower()
df_delay.head()

,BizDomainId,ActivityDate,AvgActivitiyDeal,#DelaidActivities,multiply
0,50052c47-0d0f-44c3-a13e-9ca01208e926,7/5/2022,15,10,150
1,ae20488e-2474-44d7-a90e-03743d3f9d13,6/8/2022,19,1,19
2,c919ec55-8d8b-4f47-a9bf-a893fa2ac6d8,5/14/2022,40,34,1360
3,d9d79cdf-a91a-4eb1-b6ac-aaf9a052518d,7/23/2022,9,39,351
4,2afe997d-30f0-4488-98d0-32e2bc5431cb,5/31/2022,40,38,1520


In [12]:
df_delay = df_delay.groupby('BizDomainId')[['multiply','#DelaidActivities']].sum().reset_index()
df_delay['AvgDelayTimeActivity'] = df_delay['multiply']/df_delay['#DelaidActivities']
df_final = pd.merge(df_final, df_delay[['BizDomainId','AvgDelayTimeActivity']] , how='left')
df_final.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers,#Activities,ActivitiesWoutDeal/#Activities,AvgDelayTimeActivity
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000,NaN,NaN,NaN
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397,30.0,0.866667,98.550000
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806,NaN,NaN,NaN
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307,302.0,0.662252,31.541667
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000,NaN,NaN,NaN


# Feature 6: TotalNumberOfActivities/TotlaNumberOfDeals (Assuming that the activities are all for newly registered deals)

In [13]:
df = pd.read_csv(r"User-DealWOutActivity.csv")
df['BizDomainId'] = df['BizDomainId'].str.lower()
temp = df.groupby('BizDomainId')[['NumberOfDealsOnTheDate','NumberOfActivitiesOnTheDate']].sum().reset_index()
temp['#Activities/#Deals'] =temp['NumberOfActivitiesOnTheDate']/temp['NumberOfDealsOnTheDate']

df_final = pd.merge(df_final, temp[['BizDomainId' ,'#Activities/#Deals','NumberOfDealsOnTheDate']].rename(
    columns={'NumberOfDealsOnTheDate':'#Deals'}),how='left')
df_final['#Activities/#Deals'] =df_final['#Activities/#Deals'].replace(np.inf , 0) 

In [14]:
df_final.head(10)

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers,#Activities,ActivitiesWoutDeal/#Activities,AvgDelayTimeActivity,#Activities/#Deals,#Deals
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000,NaN,NaN,NaN,NaN,0
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397,30.0,0.866667,98.550000,1.068966,29
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806,NaN,NaN,NaN,NaN,0
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307,302.0,0.662252,31.541667,0.688889,90
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000,NaN,NaN,NaN,NaN,0
5,003a5023-c29c-47d7-b484-978733878bff,NaN,others,others,1.000000,0.000000,NaN,NaN,NaN,NaN,0
6,003d7420-f7c2-4780-9bff-b893620abf5a,NaN,others,others,0.500000,0.000000,NaN,NaN,NaN,NaN,0
7,00428596-e168-49ce-8428-7949cd2215ea,0.822599,0.0,0.0,1.000000,0.949580,13.0,0.000000,51.000000,1.444444,9
8,004506ef-869d-42d6-90fb-9290ec5e0c0b,NaN,others,others,0.200000,0.000000,NaN,NaN,NaN,NaN,0
9,0049986f-b17d-44ef-992f-88d478dd21b8,NaN,others,others,1.000000,0.211382,1.0,1.000000,41.000000,0.000000,0


# Feature 7: PercentageOfNoAcquintace

In [15]:
df_file_checking['RegisterTimeDeal'] = pd.to_datetime(df_file_checking['RegisterTimeDeal'] , errors='coerce')
starttime = pd.Timestamp('2022-2-21')
mask = df_file_checking['RegisterTimeDeal']>=starttime
df_temp_deal = df_file_checking.loc[mask , ['BizDomainId' , 'DealId']].groupby(
    'BizDomainId').nunique().reset_index().rename(columns={'DealId':'#Deals_file'})
df_final = pd.merge(df_final , df_temp_deal , how='left')

In [16]:
df_noacquintance = pd.read_csv(r"NoAcquintance.csv")
df_noacquintance.head()

df_noacquintance['BizDomainId'] = df_noacquintance['BizDomainId'].str.lower()
df_noacquintance['temp']= df_noacquintance['#Deals']*df_noacquintance['%NoAcquintance']
temp = df_noacquintance.groupby('BizDomainId')['temp'].sum().reset_index()
df_final = pd.merge(df_final , temp[['BizDomainId', 'temp']] , how='left')
df_final['%NoAcquitance'] = df_final['temp']/df_final['#Deals_file']
del df_temp_deal
del df_final['temp']

# without deal-> without acquintnace
mask = (pd.isna(df_final['#Deals_file'])) & (pd.isna(df_final['%NoAcquitance']))
df_final.loc[mask , '%NoAcquitance'] = 'without_deal'

# with deal -> full acquiantace
mask = (~pd.isna(df_final['#Deals_file'])) & (pd.isna(df_final['%NoAcquitance']))
df_final.loc[mask ,'%NoAcquitance'] = 0
df_final.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers,#Activities,ActivitiesWoutDeal/#Activities,AvgDelayTimeActivity,#Activities/#Deals,#Deals,#Deals_file,%NoAcquitance
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000,NaN,NaN,NaN,NaN,0,NaN,without_deal
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397,30.0,0.866667,98.550000,1.068966,29,29.0,0.310345
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806,NaN,NaN,NaN,NaN,0,NaN,without_deal
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307,302.0,0.662252,31.541667,0.688889,90,97.0,0.917526
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000,NaN,NaN,NaN,NaN,0,NaN,without_deal


# Feature 8: Transformation Speed

In [17]:
df_trans_speed = pd.read_csv(r"Transformation Speed.csv")
df_trans_speed['BizDomainId'] = df_trans_speed['BizDomainId'].str.lower()
df_trans_speed['DateDeal']=pd.to_datetime(df_trans_speed['DateDeal'] , errors='coerce')

starttime = pd.Timestamp('2022-2-21')
finishtime=pd.Timestamp('2023')
df_trans_speed['DateDeal'] = np.where((df_trans_speed['DateDeal']>=starttime) & (df_trans_speed['DateDeal']<finishtime),
                                     df_trans_speed['DateDeal'] , pd.Timestamp('nat'))
df_trans_speed = df_trans_speed.dropna()
df_trans_speed['temp']= df_trans_speed['#SuccessfulDeals']*df_trans_speed['Speed']
temp = df_trans_speed.groupby('BizDomainId')[['temp','#SuccessfulDeals']].sum().reset_index()
temp['TransformationSpeed'] =temp['temp']/temp['#SuccessfulDeals']
df_final = pd.merge(df_final, temp[['BizDomainId' ,'TransformationSpeed']],how='left')
df_final.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers,#Activities,ActivitiesWoutDeal/#Activities,AvgDelayTimeActivity,#Activities/#Deals,#Deals,#Deals_file,%NoAcquitance,TransformationSpeed
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000,NaN,NaN,NaN,NaN,0,NaN,without_deal,NaN
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397,30.0,0.866667,98.550000,1.068966,29,29.0,0.310345,10.777778
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806,NaN,NaN,NaN,NaN,0,NaN,without_deal,6.857143
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307,302.0,0.662252,31.541667,0.688889,90,97.0,0.917526,14.000000
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000,NaN,NaN,NaN,NaN,0,NaN,without_deal,NaN


# Feature 9: Transformation rate

In [18]:
df_trans_rate = pd.read_csv(r"Transformation rate.csv")
df_trans_rate['BizDomainId'] = df_trans_rate['BizDomainId'].str.lower()
df_trans_rate['temp']= df_trans_rate['#TotalDeals']*df_trans_rate['TransformationRate']
temp = df_trans_rate.groupby('BizDomainId')[['temp','#TotalDeals']].sum().reset_index()
temp['TransformationRate'] =temp['temp']/temp['#TotalDeals']
df_final = pd.merge(df_final, temp[['BizDomainId' ,'TransformationRate']],how='left')
df_final.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers,#Activities,ActivitiesWoutDeal/#Activities,AvgDelayTimeActivity,#Activities/#Deals,#Deals,#Deals_file,%NoAcquitance,TransformationSpeed,TransformationRate
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000,NaN,NaN,NaN,NaN,0,NaN,without_deal,NaN,NaN
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397,30.0,0.866667,98.550000,1.068966,29,29.0,0.310345,10.777778,0.400000
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806,NaN,NaN,NaN,NaN,0,NaN,without_deal,6.857143,NaN
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307,302.0,0.662252,31.541667,0.688889,90,97.0,0.917526,14.000000,0.555556
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000,NaN,NaN,NaN,NaN,0,NaN,without_deal,NaN,NaN


# Feature 10: Without Reason of failure

In [19]:
df_reason = pd.read_csv(r"WithoutReasonofFailure.csv")
df_reason['BizDomainId'] = df_reason['BizDomainId'].str.lower()
df_reason['temp']= df_reason['%NoReasonFailure']*df_reason['#Failure']
temp = df_reason.groupby('BizDomainId')[['temp','#Failure']].sum().reset_index()
temp['%NoReasonFailure'] =temp['temp']/temp['#Failure']
df_final = pd.merge(df_final, temp[['BizDomainId' ,'%NoReasonFailure']],how='left')

mask = (pd.isna(df_final['%NoReasonFailure'])) & (~pd.isna(df_final['TransformationRate']))
df_final.loc[mask , '%NoReasonFailure'] = 0

In [20]:
df_final.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers,#Activities,ActivitiesWoutDeal/#Activities,AvgDelayTimeActivity,#Activities/#Deals,#Deals,#Deals_file,%NoAcquitance,TransformationSpeed,TransformationRate,%NoReasonFailure
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000,NaN,NaN,NaN,NaN,0,NaN,without_deal,NaN,NaN,NaN
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397,30.0,0.866667,98.550000,1.068966,29,29.0,0.310345,10.777778,0.400000,0.0
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806,NaN,NaN,NaN,NaN,0,NaN,without_deal,6.857143,NaN,NaN
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307,302.0,0.662252,31.541667,0.688889,90,97.0,0.917526,14.000000,0.555556,0.8
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000,NaN,NaN,NaN,NaN,0,NaN,without_deal,NaN,NaN,NaN


# Feature 11: Calculating the number of ongoing Deals

In [21]:
biz1=pd.read_csv(r"Bizdomain1.csv")
biz2=pd.read_csv(r"Bizdomain2.csv")

# complete bizdomains (appending 2 sheets of Bizdomain excel)
biz=pd.concat([biz1,biz2])
temp = pd.merge(biz , df_file[['DealId' , 'StatusDeal']], how='left',left_on='DealId', right_on='DealId')
temp['0?'] = temp['StatusDeal']==0
temp = temp.groupby('BizDomainId')[['StatusDeal','0?']].agg(
    ['count' , 'sum']).loc[: ,[('StatusDeal', 'count'),('0?','sum')] ].reset_index()
temp.columns = ['BizDomainId','#Total(Correct)Deals' , '#OngoingDeals']
temp['#OngoingDeals/#Total(Correct)Deals'] = temp['#OngoingDeals']/temp['#Total(Correct)Deals']

In [22]:
df_final = pd.merge(df_final, temp[['BizDomainId' ,'#OngoingDeals/#Total(Correct)Deals']],how='left')
mask = (pd.isna(df_final['#OngoingDeals/#Total(Correct)Deals'])) & (~pd.isna(df_final['#Deals_file']))
df_final.loc[mask , '#OngoingDeals/#Total(Correct)Deals']=0
df_final.head()

,BizDomainId,no_warning_sign,BizIsExpired?,BizActive?,AddedUsers/BoughtUser,ActiveUsers/AddedUsers,#Activities,ActivitiesWoutDeal/#Activities,AvgDelayTimeActivity,#Activities/#Deals,#Deals,#Deals_file,%NoAcquitance,TransformationSpeed,TransformationRate,%NoReasonFailure,#OngoingDeals/#Total(Correct)Deals
0,000016f3-7592-40d0-8666-1ef90f518e9a,NaN,others,others,0.066667,0.000000,NaN,NaN,NaN,NaN,0,NaN,without_deal,NaN,NaN,NaN,NaN
1,0006df37-709c-4121-be9e-5210adc6a044,0.000000,1.0,0.0,0.642724,0.181397,30.0,0.866667,98.550000,1.068966,29,29.0,0.310345,10.777778,0.400000,0.0,0.500000
2,000fe9a8-2193-43bc-82f7-e10236ad2316,NaN,others,others,1.000000,0.238806,NaN,NaN,NaN,NaN,0,NaN,without_deal,6.857143,NaN,NaN,NaN
3,002d52c3-6c10-41a5-a4b2-91422a1ab307,0.238875,0.0,1.0,0.398507,0.274307,302.0,0.662252,31.541667,0.688889,90,97.0,0.917526,14.000000,0.555556,0.8,0.627451
4,00324c62-b925-43a1-a944-8bdc3f97eef3,NaN,others,others,0.166667,0.000000,NaN,NaN,NaN,NaN,0,NaN,without_deal,NaN,NaN,NaN,NaN


# Changing the ActivitiesWoutDeal/#Activities	for bizdomains without any activities

In [23]:
mask1 = (~pd.isna(df_final['#Activities'])) & (pd.isna(df_final['ActivitiesWoutDeal/#Activities']))
df_final.loc[mask1,'ActivitiesWoutDeal/#Activities'] = 'activity=0'

# Changing the AvgDelayTimeActivity for bizdomains without any activities and those who did their activities on time

In [24]:
# zero number of activities -> no delay time
mask1 = (df_final['ActivitiesWoutDeal/#Activities']=='activity=0') & (pd.isna(df_final['AvgDelayTimeActivity']))
df_final.loc[mask1,'AvgDelayTimeActivity'] = 'activity=0'

# Activies done on time
mask2 = (~pd.isna(df_final['#Activities'])) & (pd.isna(df_final['AvgDelayTimeActivity']))
df_final.loc[mask2,'AvgDelayTimeActivity']=0

# Changing dtype

In [25]:
df_final['AvgDelayTimeActivity'] = df_final['AvgDelayTimeActivity'].astype('float64')
df_final['ActivitiesWoutDeal/#Activities']=df_final['ActivitiesWoutDeal/#Activities'].astype('float64')

# Exporting Dataset

In [26]:
del df_final['#Deals']
df_final.to_csv('df_final_train.csv',index=0)